<a href="https://colab.research.google.com/github/anupojuharshita/Lane-Detection/blob/main/road%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture('/content/road.mp4')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_input = cap.get(cv2.CAP_PROP_FPS)

out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps_input, (width, height))

prev_left_lane = None
prev_right_lane = None

def region_of_interest(img):
    mask = np.zeros_like(img)
    h, w = img.shape[:2]
    polygon = np.array([[
        (0, h),
        (int(0.4 * w), int(0.55 * h)),
        (int(0.6 * w), int(0.55 * h)),
        (w, h)
    ]], np.int32)
    cv2.fillPoly(mask, polygon, 255)
    return cv2.bitwise_and(img, mask)

def average_slope_intercept(lines, img_shape, side='left'):
    x_coords, y_coords = [], []
    for x1, y1, x2, y2 in lines:
        slope = (y2 - y1) / (x2 - x1 + 1e-5)
        if side == 'left' and slope < -0.3:
            x_coords += [x1, x2]
            y_coords += [y1, y2]
        elif side == 'right' and slope > 0.3:
            x_coords += [x1, x2]
            y_coords += [y1, y2]
    if len(x_coords) == 0:
        return None
    poly = np.polyfit(y_coords, x_coords, deg=1)
    y1 = img_shape[0]
    y2 = int(img_shape[0] * 0.55)
    x1 = int(np.polyval(poly, y1))
    x2 = int(np.polyval(poly, y2))
    return (x1, y1, x2, y2)

def calculate_deviation(left_lane, right_lane, width):
    if left_lane and right_lane:
        lane_center = (left_lane[2] + right_lane[2]) / 2
        car_center = width / 2
        deviation = (lane_center - car_center) / car_center * 100
        return deviation
    return 0

def draw_overlay(frame, deviation, fps, left_detected, right_detected):
    overlay = frame.copy()
    lane_status = "Good Lane Keeping" if left_detected and right_detected else "Lane Detection Issue"
    upcoming_road = "Left Curve Ahead" if deviation > 5 else "Right Curve Ahead" if deviation < -5 else "Straight Road"

    cv2.putText(overlay, f"Deviation: {deviation:.2f}%", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(overlay, f"FPS: {fps:.2f}", (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    cv2.putText(overlay, lane_status, (50, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if left_detected and right_detected else (0, 0, 255), 2)
    cv2.putText(overlay, f"Upcoming Road: {upcoming_road}", (50, 140), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    return overlay

def draw_lane_lines(img, lines, prev_left, prev_right):
    left_lines, right_lines = [], []
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                slope = (y2 - y1) / (x2 - x1 + 1e-5)
                if slope < -0.3:
                    left_lines.append((x1, y1, x2, y2))
                elif slope > 0.3:
                    right_lines.append((x1, y1, x2, y2))

    left_lane = average_slope_intercept(left_lines, img.shape, 'left') or prev_left
    right_lane = average_slope_intercept(right_lines, img.shape, 'right') or prev_right

    if left_lane:
        cv2.line(img, (left_lane[0], left_lane[1]), (left_lane[2], left_lane[3]), (0, 255, 0), 2)
    if right_lane:
        cv2.line(img, (right_lane[0], right_lane[1]), (right_lane[2], right_lane[3]), (0, 0, 255), 2)

    if left_lane and right_lane:
        pts = np.array([[left_lane[0], left_lane[1]],
                        [left_lane[2], left_lane[3]],
                        [right_lane[2], right_lane[3]],
                        [right_lane[0], right_lane[1]]], np.int32)
        cv2.fillPoly(img, [pts], (144, 238, 144))  # Light green fill

    return img, left_lane, right_lane

# Process video
prev_time = time.time()
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    start_time = time.time()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    roi = region_of_interest(edges)
    lines = cv2.HoughLinesP(roi, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=150)

    processed, prev_left_lane, prev_right_lane = draw_lane_lines(frame.copy(), lines, prev_left_lane, prev_right_lane)

    deviation = calculate_deviation(prev_left_lane, prev_right_lane, width)
    fps = 1 / (time.time() - start_time + 1e-5)

    overlay = draw_overlay(processed, deviation, fps, prev_left_lane is not None, prev_right_lane is not None)
    out.write(overlay)

cap.release()
out.release()
print("Output saved as output.mp4")


✅ Full dual-lane adaptive detection with overlay complete. Output saved as output.mp4


In [16]:
from google.colab import files
files.download('output.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>